Model Pipeline

Input:
    - Satellite tracking data
    - Satellite slr signal data
    - Station sat-imagery weather data
Output:
    - Record of ALL satellite passes over station, weather a signal was made in that pass, and what the cloud conditons were like at the time of pass


In [1]:
# this script generates a reort on the cloud and slr da6ta in line of sight of the stattion
import pandas as pd
import pandas_profiling
import pandas as pd
import datetime 
from datetime import datetime
from datetime import time
import os

/var/folders/py/74k69jjd0y74r977spsdpz540000gn/T/ipykernel_746/2565732241.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [3]:
def get_cloud_data(station_code, sat_name, sat_id, start_date, end_date):

    # read from file cloud and slr data
    

    # 3   MERGE PASS AND CLOUD DATA - SAVE TO FILE

    merged_df = pd.merge(slr_merged_df, cloud_df, on='time', how='inner')
    merged_df = merged_df[['time', 'mean_receive_amp',  'std_receive_amp', 'cloud_cov']]
    merged_df.to_csv(f"/Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/exploration_data_analysis/data/{station_code}/satellites/{sat_name}_2022-08_2023-08_merged.txt", sep='\t')

    cloud_slr_df = pd.read_csv(f'/Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/exploration_data_analysis/data/{station_code}/satellites/{sat_name}_2022-08_2023-08_merged.txt', sep='\t')
    cloud_slr_df = cloud_slr_df[['time', 'mean_receive_amp', 'cloud_cov']].copy()

    cloud_slr_df['time'] = pd.to_datetime(cloud_slr_df['time'])
    cloud_slr_filtered_df = cloud_slr_df[(cloud_slr_df['time'] >= start_date) & (cloud_slr_df['time'] < end_date)]

    return filtered_df 

In [4]:
def los_signal_true_false(passes_filtered_df, cloud_slr_filtered_df, sat_name, station_code):
    los_data = []

    for i in range(0, len(passes_filtered_df), 2):
        try:
            los_start = passes_filtered_df['time'].iloc[i].strftime('%Y-%m-%d %H:%M:%S')
            los_end = passes_filtered_df['time'].iloc[i + 1].strftime('%Y-%m-%d %H:%M:%S')
        except IndexError:
            print("IndexError: No end of pass marker at end_date. Skipping pass...")
            continue

        stage_cloud_slr = cloud_slr_filtered_df[
            (cloud_slr_filtered_df['time'] >= los_start) & (cloud_slr_filtered_df['time'] <= los_end)
        ]

        average_cloud_cov = stage_cloud_slr['cloud_cov'].mean()

        los_data.append({
            'station': station_code,
            'satellite': sat_name,
            'pass_start_date': los_start,
            'pass_end_date': los_end,
            'mean_cloud': average_cloud_cov,
        })

    los_df = pd.DataFrame(los_data)
    return los_df


In [ ]:
def read_turbulence_data(fits_file, lat, lon):
    with fits.open(fits_file) as hdul:
        data = hdul[0].data

        # Define the geographical bounds
        lat_min = -90.0
        lat_max = 90.0
        lon_min = -180.0
        lon_max = 180.0
        
        # Get the resolution of the data
        lat_res = (lat_max - lat_min) / data.shape[0]
        lon_res = (lon_max - lon_min) / data.shape[1]

        # Find the nearest grid point to the requested latitude and longitude
        lat_idx = int((lat - lat_min) / lat_res)
        lon_idx = int((lon - lon_min) / lon_res)

        # Ensure the indices are within the data bounds
        lat_idx = np.clip(lat_idx, 0, data.shape[0] - 1)
        lon_idx = np.clip(lon_idx, 0, data.shape[1] - 1)

        # Get the turbulence strength value at the specified location
        turbulence_strength = data[lat_idx, lon_idx]

        return turbulence_strength

In [5]:
import os
import pandas as pd
import gc
from contextlib import ExitStack
from tqdm import tqdm  # Import tqdm for progress bars


def main():
    start_date = '2022-08-01' # START TIME OF SIMULATION
    end_date = '2023-08-01' # END OF SIMULATION

    # REQUIRMENTS MAREN AND KAREN add here more
    ogs_latitude = 30.7128  # Example latitude for New York City
    ogs_longitude = -74.0060  # Example longitude for New York City

    # GET SATELLITE PASSES OVER EACH OGS
    file_path = f"PATH TO KARENS SAT PASSES FOR EACH OGS TXT FILES FOR TIMEFRAME"
    passes_unfil_df = pd.read_csv(file_path, sep='\t', parse_dates=['time'])
    passes_df = passes_unfil_df[(passes_unfil_df['time'] >= start_date) & (passes_unfil_df['time'] < end_date)]

    # GET CLOUD OVER EACH OGS
    cloud_df = pd.read_csv('FILE WITH OGS CLOUD DATA FOR TIMEFRAME', sep='\t')
    cloud_df['time'] = pd.to_datetime(cloud_df['time'], format = "%Y%m%d%H%M%S")
    cloud_df = cloud_df.sort_values(by='time', ascending=True)
    cloud_df = cloud_df[['time', 'cloud_cov']]
    cloud_df.reset_index(drop=True, inplace=True)

    # A - MERGE CLOUD WITH SATELLITE PASSES FOR EACH OGS

    # GET TURBULANCE OVER EACH OGS PER PASS
    fits_file_day = 'PATH TO TURBULANCE FITS FILE DAY'
    fits_file_night = 'PATH TO TURBULANCE FITS FILE NIGHT'
    ogs_turbulence_strength_day = read_turbulence_data(fits_file_day, ogs_latitude, ogs_longitude)
    ogs_turbulence_strength_night = read_turbulence_data(fits_file_night, ogs_latitude, ogs_longitude)
    #print(f"Turbulence strength during the day at location ({ogs_latitude}, {ogs_longitude}): {ogs_turbulence_strength_day}")
    #print(f"Turbulence strength during the night at location ({ogs_latitude}, {ogs_longitude}): {ogs_turbulence_strength_night}")

    # B - ADD THE TURBULANCE DATA TO A (based if the time of pass is night or day)

    # STATIC PASS add call for isaac code

    # C - ADD THE DATA TRANSFERED FOR EACH SAT PASS IN B (data = 0 where turbulance strength high)

    # CALCULATE NETWORK PERFORMANCE


if __name__ == "__main__":
    main()


        

Processing data with elevation angle 45


Satellites:   0%|          | 0/35 [00:00<?, ?satellite/s, Current Satellite=galileo210]

Satellites: 100%|██████████| 35/35 [38:57<00:00, 66.79s/satellite, Current Satellite=galileo209]


Model for dataframe finished processing: see /Users/eugenerotherham/Documents/AtherasAnalytics/clouds-outage-prediction-main/exploration_data_analysis/pipeline_results/elev45_model_df.txt
